In [3]:
from IPython.display import display, Math

latex_code = r"""
\begin{aligned}
\textbf{Decision Variables:} \quad
&c_{\text{chain}}, \; c_A, \; c_B, \; m_{\text{chain}}, \; m_A, \; m_B, \\
&u_{\text{chain}}, \; u_A, \; u_B, \; a_{\text{chain}}, \; a_A, \; a_B \;\;(\text{all integers, } \ge 0). \\[6pt]

\textbf{Volume (kg):} \quad
V_{\text{chain}} &= 1000 + 500 d_{\text{chain}} + 10 c_{\text{chain}} + 15 m_{\text{chain}} + 5 u_{\text{chain}} + 15 a_{\text{chain}}, \\
V_A &= 1000 + 2000 d_A + 5 c_A + 5 m_A + 2 u_A + 5 a_A, \\
V_B &= 1000 + 10000 d_B + 0.5 c_B + 1 m_B + 1 u_B + 3 a_B. \\[6pt]

\textbf{Revenue:} \quad
P_{\text{chain}} &= 150(1 - d_{\text{chain}}) V_{\text{chain}}, \\
P_A &= 180(1 - d_A) V_A, \\
P_B &= 200(1 - d_B) V_B, \\
P_{\text{total}} &= P_{\text{chain}} + P_A + P_B. \\[6pt]

\textbf{Cost:} \quad
C_{\text{chef}} &= 500 (c_{\text{chain}} + c_A + c_B), \\
C_{\text{macro}} &= 100000 (m_{\text{chain}} + m_A + m_B), \\
C_{\text{micro}} &= 30000 (u_{\text{chain}} + u_A + u_B), \\
C_{\text{digital}} &= 150000 a_{\text{chain}} + 50000 a_A + 5000 a_B, \\
C_{\text{total}} &= C_{\text{chef}} + C_{\text{macro}} + C_{\text{micro}} + C_{\text{digital}}. \\[6pt]

\textbf{Objective:} \quad
&\max V_{\text{total}}, \quad \text{where} \quad V_{\text{total}} = V_{\text{chain}} + V_A + V_B. \\[6pt]

\textbf{Constraints:} \quad
&C_{\text{total}} \leq 500000, \\
&c_{\text{chain}} + c_A + c_B \leq 30. \\
\end{aligned}
"""

display(Math(latex_code))


<IPython.core.display.Math object>

In [4]:
import pulp

def solve_scenario(d_chain_val, d_A_val, d_B_val):
    """
    Solve the chef/influencer/digital marketing model to MAXIMIZE VOLUME.
    Additionally computes Revenue and Profit (Revenue - Cost).
    """
    # Create an LP Problem
    prob = pulp.LpProblem("Maximize_Volume", pulp.LpMaximize)  # Maximization
    
    # Define Decision Variables
    c_chain = pulp.LpVariable('c_chain', lowBound=0, upBound=100, cat='Integer')
    c_A = pulp.LpVariable('c_A', lowBound=0, upBound=1000, cat='Integer')
    c_B = pulp.LpVariable('c_B', lowBound=0, upBound=10000, cat='Integer')
    
    m_chain = pulp.LpVariable('m_chain', lowBound=0, upBound=100, cat='Integer')
    m_A = pulp.LpVariable('m_A', lowBound=0, upBound=1000, cat='Integer')
    m_B = pulp.LpVariable('m_B', lowBound=0, upBound=10000, cat='Integer')
    
    u_chain = pulp.LpVariable('u_chain', lowBound=0, upBound=100, cat='Integer')
    u_A = pulp.LpVariable('u_A', lowBound=0, upBound=1000, cat='Integer')
    u_B = pulp.LpVariable('u_B', lowBound=0, upBound=10000, cat='Integer')
    
    a_chain = pulp.LpVariable('a_chain', lowBound=0, upBound=100, cat='Integer')
    a_A = pulp.LpVariable('a_A', lowBound=0, upBound=1000, cat='Integer')
    a_B = pulp.LpVariable('a_B', lowBound=0, upBound=10000, cat='Integer')
    
    # Compute Volume (kg) -- using discount factors as volume boosters
    chain_discount_volume = 500 * d_chain_val
    A_discount_volume = 2000 * d_A_val
    B_discount_volume = 10000 * d_B_val
    
    volume_chain = (1000 
                    + chain_discount_volume
                    + 10*c_chain
                    + 15*m_chain
                    + 5*u_chain
                    + 15*a_chain)
    volume_A = (1000
                + A_discount_volume
                + 5*c_A
                + 5*m_A
                + 2*u_A
                + 5*a_A)
    volume_B = (1000
                + B_discount_volume
                + 0.5*c_B
                + 1*m_B
                + 1*u_B
                + 3*a_B)
    total_volume = volume_chain + volume_A + volume_B
    
    # Costs
    cost_chef = 500*(c_chain + c_A + c_B)
    cost_macro = 100000*(m_chain + m_A + m_B)
    cost_micro = 30000*(u_chain + u_A + u_B)
    cost_digital = 150000*a_chain + 50000*a_A + 5000*a_B
    total_cost = cost_chef + cost_macro + cost_micro + cost_digital
    
    # Constraints
    prob += total_cost <= 500000, "BudgetConstraint"   # Hard budget limit
    prob += c_chain + c_A + c_B <= 30, "ChefVisitsConstraint"
    
    # Objective: Maximize total_volume
    prob += total_volume
    
    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    
    # Collect results
    status = pulp.LpStatus[prob.status]
    
    # Get volumes from the optimized solution
    chain_vol_val = pulp.value(volume_chain)
    A_vol_val = pulp.value(volume_A)
    B_vol_val = pulp.value(volume_B)
    
    # Compute revenue based on discounted price
    chain_revenue = chain_vol_val * 150 * (1 - d_chain_val)
    A_revenue = A_vol_val * 180 * (1 - d_A_val)
    B_revenue = B_vol_val * 200 * (1 - d_B_val)
    total_revenue = chain_revenue + A_revenue + B_revenue
    
    # Profit = Revenue - Cost
    total_cost_val = pulp.value(total_cost)
    profit = total_revenue - total_cost_val
    
    solution = {
        'Status': status,
        'TotalVolume': pulp.value(total_volume),
        'TotalCost': total_cost_val,
        'RevenueChain': chain_revenue,
        'RevenueA': A_revenue,
        'RevenueB': B_revenue,
        'TotalRevenue': total_revenue,
        'Profit': profit,
        'c_chain': c_chain.varValue,
        'c_A': c_A.varValue,
        'c_B': c_B.varValue,
        'm_chain': m_chain.varValue,
        'm_A': m_A.varValue,
        'm_B': m_B.varValue,
        'u_chain': u_chain.varValue,
        'u_A': u_A.varValue,
        'u_B': u_B.varValue,
        'a_chain': a_chain.varValue,
        'a_A': a_A.varValue,
        'a_B': a_B.varValue,
    }
    return status, solution

In [5]:
# Test discount combinations (0% to 40% for all segments)
best_volume = -float('inf')
best_solution = None
best_discounts = None

for d_chain in [0.0, 0.1, 0.2, 0.3, 0.4]:
    for d_A in [0.0, 0.1, 0.2, 0.3, 0.4]:
        for d_B in [0.0, 0.1, 0.2, 0.3, 0.4]:
            status, solution = solve_scenario(d_chain, d_A, d_B)
            if status == 'Optimal':
                if solution['TotalVolume'] > best_volume:
                    best_volume = solution['TotalVolume']
                    best_solution = solution
                    best_discounts = (d_chain, d_A, d_B)

# Print results
if best_solution:
    print("BEST SCENARIO DISCOUNTS:")
    print(f"  Chain discount = {best_discounts[0]*100:.0f}%")
    print(f"  A discount     = {best_discounts[1]*100:.0f}%")
    print(f"  B discount     = {best_discounts[2]*100:.0f}%")
    
    print("\nRESULTS:")
    print(f"Total Volume: {best_solution['TotalVolume']:.2f} kg")
    print(f"Total Cost: {best_solution['TotalCost']:.2f} TL")
    
    print("\nREVENUES (after discount):")
    print(f"  Chain Revenue: {best_solution['RevenueChain']:.2f} TL")
    print(f"  A Revenue:     {best_solution['RevenueA']:.2f} TL")
    print(f"  B Revenue:     {best_solution['RevenueB']:.2f} TL")
    print(f"Total Revenue:    {best_solution['TotalRevenue']:.2f} TL")
    
    print(f"\nPROFIT (Revenue - Cost): {best_solution['Profit']:.2f} TL")
    
    print("\nACTIVITIES:")
    print(f"Chef Visits - Chain: {best_solution['c_chain']}, A: {best_solution['c_A']}, B: {best_solution['c_B']}")
    print(f"Macro - Chain: {best_solution['m_chain']}, A: {best_solution['m_A']}, B: {best_solution['m_B']}")
    print(f"Micro - Chain: {best_solution['u_chain']}, A: {best_solution['u_A']}, B: {best_solution['u_B']}")
    print(f"Digital Ads - Chain: {best_solution['a_chain']}, A: {best_solution['a_A']}, B: {best_solution['a_B']}")
else:
    print("NO FEASIBLE SOLUTION FOUND")

BEST SCENARIO DISCOUNTS:
  Chain discount = 40%
  A discount     = 40%
  B discount     = 40%

RESULTS:
Total Volume: 8591.00 kg
Total Cost: 500000.00 TL

REVENUES (after discount):
  Chain Revenue: 135000.00 TL
  A Revenue:     194400.00 TL
  B Revenue:     634920.00 TL
Total Revenue:    964320.00 TL

PROFIT (Revenue - Cost): 464320.00 TL

ACTIVITIES:
Chef Visits - Chain: 30.0, A: 0.0, B: 0.0
Macro - Chain: 0.0, A: 0.0, B: 0.0
Micro - Chain: 0.0, A: 0.0, B: 0.0
Digital Ads - Chain: 0.0, A: 0.0, B: 97.0
